# 05. Depth Charts
Source: <br>
1. MLB.com team depth charts <br>
2. Wayback Machine for past games <br>

Description: This scrapes bullpen depth charts and infers leverage from order <br>

### Bullpen Depth Charts

In [10]:
# This scrapes the bullpen depth chart for teams via their website or via the Wayback Machine
# Top reliever will be the closer. Usually other high-leverage pitchers will be near top
# Need this header to trick site into thinking this isn't a scrape
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
    }

# Extract bullpen information from MLB.com depth charts
def scrape_bullpen(url, header, abbrev):
    # Get data from URL
    r = requests.get(url, headers=header)
    dfs = pd.read_html(r.text, encoding='iso-8859-1')
    # Bullpen can be one of two tables
    try:
        df = dfs[2]
        # Remove if they're on IL
        df = df[df["Bullpen.1"].str.contains("IL-")==False].reset_index()
        # Or in the minors
        df = df[df["Bullpen.1"].str.contains(" Minors")==False].reset_index()
    except:
        df = dfs[1]
        df = df[df["Bullpen.1"].str.contains("IL-")==False].reset_index()  
        df = df[df["Bullpen.1"].str.contains(" Minors")==False].reset_index()
        
    # Assume leverage is 0 (these pitchers will never come into a game, reserved for SPs on days off)
    df['Leverage'] = 0
    # Loop through rows
    for i in range(len(df)):
        # The top guy should be the closer
        if i == 0:
            df['Leverage'][i] = 4
        # Then the next five are set up/high leverage
        elif i < 4:
            df['Leverage'][i] = 3
        # Then low leverage
        elif i < 11:
            df['Leverage'][i] = 2

    # Extract name from column Bullpen.1
    df[['Name', 'drop']] = df['Bullpen.1'].str.split("B/T", expand=True)
    # Remove numbers
    df['Name'] = df['Name'].str.replace('\d+', '')
    # Remove closer tag
    df['Name'] = df['Name'].str.replace("\(CL\)", '')
    
    # Clean name
    df['Name'] = df.apply(lambda x: remove_accents(x['Name']), axis=1)  # remove accents
    df['Name'] = df['Name'].str.strip()
    
    # Keep Name, Bats/Throws, Leverage
    df = df[['Name', 'B/T', 'Leverage']]
    
    return df

### All Depth Charts

In [1]:
# This loops over teams and scrapes all depth charts
def create_depth_charts(start_date, end_date):
    # Date range
    start_date = datetime.datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.datetime.strptime(end_date, "%Y%m%d")
    delta = datetime.timedelta(days=1)
    
    # Loop over dates
    while start_date <= end_date:
        date = start_date.strftime("%Y%m%d")
        
        # Create roster directory
        directory = "Depth" + date
        try:
            os.mkdir(os.path.join(baseball_path, "5. Depth Charts", directory))
        except:
            pass

        for index, row in team_map.iterrows():
            mlburl = row['MLBURL']
            abbrev = row['BBREFTEAM']
        
            # Wayback Machine is good for backtesting, but won't default to current date
            # url = f"https://web.archive.org/web/{date}/https://www.mlb.com/{mlburl}/roster/depth-chart"
            url = f"https://www.mlb.com/{mlburl}/roster/depth-chart"
            df = scrape_bullpen(url, header, abbrev)
        
            # filename = "Depth_Chart_" + abbrev + "_" + date + ".csv"
            df.to_csv(os.path.join(baseball_path, "5. Depth Charts", directory, f"Depth_Chart_{abbrev}_{date}.csv"), encoding='iso-8859-1')
        
        start_date += delta

Note:
Scraping past bullpens is possible using the Wayback Machine. It will provide the most recent data as of the data provided in the URL, even if the date isn't available. So if 4/3 exists and 4/4 doesn't, when you try to create the depth chart on 4/4, it'll give you the same depth chart as 4/3 <br>
When running day-of, you don't want to use the Wayback Machine as it will choose the last date it scraped <br>
May not provide a pitcher of each leverage level. Prone to missing closers on occassion. 